# Automated Capacity Analysis using Smart LAMA API
This is a notebook to test some capacity analysis operations on a simple LAMA.

**This notebook has EXECUTE access and cannot be edited. You can duplicate it and then edit it if needed.**



## Preamble

In [1]:
# We will use an API to send the model and obtain the result. 
# We need the requests and json libraries to send and parse requests and responses.

import requests, json 

# Some auxiliary constants and variables

SECONDS_PER_MONTH= 60*60*24*30
SECONDS_PER_DAY= 60*60*24
MAXIMUM_COST= 99999999.99

# api_url = 'https://smart-lama-api-beta.herokuapp.com/api/v2'
api_url = 'https://smart-lama.services.governify.io/api/v2'
urlBase= api_url+'/operations/'

## Overview

The steps necessary to perform the capacity analysis queries are:

- Modelling the architecture of the service by considering the internal services, the external APIs on which it depends, and their pricings. For short, an architecture with all this information will be called LAMA (Limitation-Aware Microservices Architecture).
- Interpreting each query as invocations to Smart LAMA API, a public API that provides basic query operations, namely:
   - maxRequests(timeWindow, maximumCost) --> maximum number of requests that the service is able to serve.
   - maxRequestsKeys(timeWindow, keysE1Basic, keysE1Premium, keysE2Silver, keysE2Gold) --> maximum number of requests that the service is able to serve having a certain number of API keys for each plan. Note that the function created in this notebook is specific to the LAMA being modelled.
   - minTime(numRequests, maximumCost) --> minimum time that the service needs to serve at least 'numRequests' requests.
   - minTime(numRequests, keysE1Basic, keysE1Premium, keysE2Silver, keysE2Gold) --> minimum time that the service needs to serve at least 'numRequests' having a certain number of API keys for each plan. Note that the function created in this notebook is specific to the LAMA being modelled.
   - minCost(numRequests, numSeconds) --> minimum cost of the service, so that it can serve a minimum of 'numRequests' over a time window of 'numSeconds' seconds.
   Note that all operations have an additional, optional parameter 'useOvg' to indicate whether overage requests can be used.


In [2]:
# Wrappers for calling the API

headers = {'Content-Type': 'application/json'}

def maxRequests(ldl,timeWindow,maximumCost=MAXIMUM_COST,useOvg="false"):
    api_q= urlBase+'maxRequests?OpEx={0}&time={1}&useOvg={2}'.format(maximumCost,timeWindow,useOvg)
    res = requests.post(api_q,data=json.dumps(ldl),headers=headers).json()
    result = res['result']
    response = res['response']
    return (result, response)

def maxRequestsKeys(ldl,timeWindow,keysE1Basic,keysE1Premium,keysE2Silver,keysE2Gold,useOvg="false"):
    api_q= urlBase+'maxRequests?K-E1-Basic={0}&K-E1-Premium={1}&K-E2-Silver={2}&K-E2-Gold={3}&time={4}&useOvg={5}'.format(keysE1Basic,keysE1Premium,keysE2Silver,keysE2Gold,timeWindow,useOvg)
    res = requests.post(api_q,data=json.dumps(ldl),headers=headers).json()
    result = res['result']
    response = res['response']
    return (result, response)

def minCost(ldl,numRequests,numSeconds=SECONDS_PER_MONTH,useOvg="false"):
    api_q= urlBase+'minCost?reqL={0}&time={1}&useOvg={2}'.format(numRequests,numSeconds,useOvg)
    res = requests.post(api_q,data=json.dumps(ldl),headers=headers).json()
    result = res['result']
    response = res['response']
    return (result, response)

def minTime(ldl,numRequests,maximumCost=MAXIMUM_COST,useOvg="false"):
    api_q= urlBase+'minTime?reqL={0}&OpEx={1}&useOvg={2}'.format(numRequests, maximumCost,useOvg)
    res = requests.post(api_q,data=json.dumps(ldl),headers=headers).json()
    result = res['result']
    response = res['response']
    return (result, response)

def minTimeKeys(ldl,numRequests,keysE1Basic,keysE1Premium,keysE2Silver,keysE2Gold,useOvg="false"):
    api_q= urlBase+'minTime?reqL={0}&K-E1-Basic={1}&K-E1-Premium={2}&K-E2-Silver={3}&K-E2-Gold={4}&useOvg={5}'.format(numRequests,keysE1Basic,keysE1Premium,keysE2Silver,keysE2Gold,useOvg)
    res = requests.post(api_q,data=json.dumps(ldl),headers=headers).json()
    result = res['result']
    response = res['response']
    return (result, response)

#### Creating the LAMA

The LAMA in this example contains 3 internal services and 2 external APIs, each one having 2 different plans. This LAMA is the same as the one in Fig. 1 of the paper.

The following cell contains a function to POST the LAMA to the API. An additional cell containing functions to delete LAMAs is included but not used in this notebook.

![Picture title](basicmodel.png)

In [3]:
# Description of the LAMA to analyse
lama_dl = {
    "services": ["S1","S2","S3"],
    "external": ["E1","E2"],
    "entry": "S1",
    "relationships": [
        {
            "from": "S1",
            "to": "S2",
            "value": 3
        },
        {
            "from": "S1",
            "to": "S3",
            "value": 2
        },
        {
            "from": "S2",
            "to": "E1",
            "value": 2
        },
        {
            "from": "S3",
            "to": "E1",
            "value": 1
        },
        {
            "from": "S3",
            "to": "E2",
            "value": 2
        }
    ],
    "pricings": [
        {
            "external": "E1",
            "plans": [
                {
                    "name": "Basic",
                    "cost": 5.0,
                    "rate": 15,
                    "rateunit": 1,
                    "quota": 1000,
                    "quotaunit": 86400,
                    "ovg": 0.01
                },
                {
                    "name": "Premium",
                    "cost": 8.0,
                    "rate": 25,
                    "rateunit": 1,
                    "quota": 10000,
                    "quotaunit": 86400
                }
            ]
        },
        {
            "external": "E2",
            "plans": [
                {
                    "name": "Silver",
                    "cost": 4.0,
                    "rate": 10,
                    "rateunit": 1
                },
                {
                    "name": "Gold",
                    "cost": 10.0,
                    "rate": 20,
                    "rateunit": 1
                }
            ]
        }
    ]
}

## Analysis Operations

In [4]:
# Q1. What is the cheapest operational cost for my LAMA in order to offer 2 RPS to 20 customers?
r=2
cstm=20
t=1
(MR,fullResponse) = minCost(lama_dl, r*cstm, t)
print ('The cheapest operational cost to offer {0} requests per {1} second(s) to {2} customers is $ {3} '.format(r,t,cstm,MR))
print (fullResponse)

The cheapest operational cost to offer 2 requests per 1 second(s) to 20 customers is $ 174 
<pre>reqS1 = 40;
reqS2 = 120;
reqS3 = 80;
reqE1 = 320;
reqE2 = 160;
reqL = 40;
time = 1;
OpEx = 174.0;
limReqE1 = array1d(PlansE1, [120, 200]);
ovgReqE1 = array1d(PlansE1, [0, 0]);
limReqE2 = array1d(PlansE2, [100, 60]);
ovgReqE2 = array1d(PlansE2, [0, 0]);
keysE1 = array1d(PlansE1, [8, 8]);
OpExE1 = 104.0;
keysE2 = array1d(PlansE2, [10, 3]);
OpExE2 = 70.0;
aux = 0;
----------
</pre>


In [5]:
# Q2. Assuming we have a Basic plan and a Gold plan already contracted what is the maximal RPM I can
# guarantee to all my 20 customers?

keysE1Basic=1
keysE1Premium=0
keysE2Silver=0
keysE2Gold=1
t=60
cstm=20
(MR,fullResponse) = maxRequestsKeys(lama_dl, t, keysE1Basic, keysE1Premium, keysE2Silver, keysE2Gold)
MR=MR/cstm
print ('The maximum number of requests that each one of {0} customers can make in {1} second(s) with the specified keys is {2} '.format(cstm,t,MR))
print (fullResponse)

The maximum number of requests that each one of 20 customers can make in 60 second(s) with the specified keys is 5.6 
<pre>reqS1 = 112;
reqS2 = 336;
reqS3 = 224;
reqE1 = 896;
reqE2 = 448;
reqL = 112;
time = 60;
OpEx = 15.0;
limReqE1 = array1d(PlansE1, [896, 0]);
ovgReqE1 = array1d(PlansE1, [0, 0]);
limReqE2 = array1d(PlansE2, [0, 448]);
ovgReqE2 = array1d(PlansE2, [0, 0]);
keysE1 = array1d(PlansE1, [1, 0]);
OpExE1 = 5.0;
keysE2 = array1d(PlansE2, [0, 1]);
OpExE2 = 10.0;
aux = 0;
----------
</pre>


In [7]:
# Q3. Assuming we have a monthly budget limit of $120 in my LAMA, which is the maximum RPS
# to each of 20 customers?

t=1
c=120
cstm=20
(MR,fullResponse) = maxRequests(lama_dl, t, c)
MR=MR/cstm
print ('The maximum number of requests that each one of {0} customers can make in {1} second(s) for less than $ {2} is {3}'.format(cstm,t,c,MR))
print (fullResponse)

The maximum number of requests that each one of 20 customers can make in 1 second(s) for less than $ 120 is 1.35
<pre>reqS1 = 27;
reqS2 = 81;
reqS3 = 54;
reqE1 = 216;
reqE2 = 108;
reqL = 27;
time = 1;
OpEx = 118.0;
limReqE1 = array1d(PlansE1, [0, 216]);
ovgReqE1 = array1d(PlansE1, [0, 0]);
limReqE2 = array1d(PlansE2, [90, 18]);
ovgReqE2 = array1d(PlansE2, [0, 0]);
keysE1 = array1d(PlansE1, [0, 9]);
OpExE1 = 72.0;
keysE2 = array1d(PlansE2, [9, 1]);
OpExE2 = 46.0;
aux = 0;
----------
</pre>


In [7]:
# Q4. What is the cheapest operational cost to guarantee a global operating condition of 50 RPS?

r=50
t=1
(MR,fullResponse) = minCost(lama_dl, r, t)
print ('The cheapest operational cost to offer {0} requests per {1} second(s) is $ {2} '.format(r,t,MR))
print (fullResponse)

The cheapest operational cost to offer 50 requests per 1 second(s) is $ 220 
<pre>reqS1 = 50;
reqS2 = 150;
reqS3 = 100;
reqE1 = 400;
reqE2 = 200;
reqL = 50;
time = 1;
OpEx = 220.0;
limReqE1 = array1d(PlansE1, [150, 250]);
ovgReqE1 = array1d(PlansE1, [0, 0]);
limReqE2 = array1d(PlansE2, [100, 100]);
ovgReqE2 = array1d(PlansE2, [0, 0]);
keysE1 = array1d(PlansE1, [10, 10]);
OpExE1 = 130.0;
keysE2 = array1d(PlansE2, [10, 5]);
OpExE2 = 90.0;
aux = 0;
----------
</pre>


In [8]:
# Q5. Assuming we have a Basic plan and a Gold plan already contracted what is
# the maximal RPS I can guarantee as operating condition?

keysE1Basic=1
keysE1Premium=0
keysE2Silver=0
keysE2Gold=1
t=1
(MR,fullResponse) = maxRequestsKeys(lama_dl, t, keysE1Basic, keysE1Premium, keysE2Silver, keysE2Gold)
print ('The maximum number of requests that can be made in {0} second(s) with the specified keys is {1} '.format(t,MR))
print (fullResponse)

The maximum number of requests that can be made in 1 second(s) with the specified keys is 1 
<pre>reqS1 = 1;
reqS2 = 3;
reqS3 = 2;
reqE1 = 8;
reqE2 = 4;
reqL = 1;
time = 1;
OpEx = 15.0;
limReqE1 = array1d(PlansE1, [8, 0]);
ovgReqE1 = array1d(PlansE1, [0, 0]);
limReqE2 = array1d(PlansE2, [0, 4]);
ovgReqE2 = array1d(PlansE2, [0, 0]);
keysE1 = array1d(PlansE1, [1, 0]);
OpExE1 = 5.0;
keysE2 = array1d(PlansE2, [0, 1]);
OpExE2 = 10.0;
aux = 0;
----------
</pre>


In [9]:
# Q6. Assuming we have a monthly budget limit of $120 in my LAMA, which is the maximum RPS
# I can guarantee as operating condition?

t=1
c=120
(MR,fullResponse) = maxRequests(lama_dl, t, c)
print ('The maximum number of requests that can be made in {0} second(s) for less than $ {1} is {2}'.format(t,c,MR))
print (fullResponse)


The maximum number of requests that can be made in 1 second(s) for less than $ 120 is 27
<pre>reqS1 = 27;
reqS2 = 81;
reqS3 = 54;
reqE1 = 216;
reqE2 = 108;
reqL = 27;
time = 1;
OpEx = 118.0;
limReqE1 = array1d(PlansE1, [0, 216]);
ovgReqE1 = array1d(PlansE1, [0, 0]);
limReqE2 = array1d(PlansE2, [90, 18]);
ovgReqE2 = array1d(PlansE2, [0, 0]);
keysE1 = array1d(PlansE1, [0, 9]);
OpExE1 = 72.0;
keysE2 = array1d(PlansE2, [9, 1]);
OpExE2 = 46.0;
aux = 0;
----------
</pre>


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=a2de20fe-81a1-45ad-b4ec-a8e29ff00187' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>